In [1]:
import numpy as np
import qiskit.pulse as pulse
from qiskit_dynamics.pulse.pulse_simulator import PulseSimulator

In [3]:
from qiskit_ibm_provider import IBMProvider
IBMProvider.save_account(token='e138ea6b754f60ffd002db17bcfc3aaf8eb7fd35f51c832aa247c8ffa6ca261b9ca3eb487d3a976b631469141fc693fdb6d37284560395483c47b27dec4a5e42')

provider = IBMProvider()
ibm_backend = provider.get_backend('ibmq_lima')

In [4]:
backend = PulseSimulator.from_backend(ibm_backend, subsystem_list=[0, 1])

In [6]:
from qiskit.pulse import library

amp = 1
sigma = 10
num_samples = 128
#%%
gaus = pulse.library.Gaussian(num_samples, amp, sigma,
                              name="Parametric Gaus")
gaus.draw()


with pulse.build() as schedule:
    pulse.play(gaus, backend.drive_channel(0))
    pulse.play(gaus, backend.drive_channel(1))
    pulse.play(gaus, backend.control_channel([0,1])[0])
    pulse.play(gaus, backend.control_channel([1,0])[0])
    pulse.play(gaus, backend.control_channel([1,2])[0])
    # Below need to fix the bug https://github.com/Qiskit/qiskit-dynamics/issues/127
    pulse.play(gaus, backend.control_channel([2,1])[0])
    pulse.play(gaus, backend.control_channel([1,3])[0])
    pulse.play(gaus, backend.control_channel([3,1])[0])
    # pulse.play(gaus, backend.drive_channel(2))
    # pulse.play(gaus, backend.drive_channel(3))
    # pulse.play(gaus, backend.drive_channel(4))

schedule.draw()
y0 = np.zeros(backend.solver.model.dim)
y0[0] = 1
t_span = np.array([0, num_samples * backend.solver._dt])
result = backend.run(schedule, y0=y0, t_span=t_span)
#%%
result

Result(backend_name='Pulse Simulator of ibmq_lima', backend_version='0.1', qobj_id='', job_id='ee01cb54-7192-449f-8cc5-11d80480a7ed', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(unitary=[[ 1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 1.00000000e+00+8.79141144e-11j  3.45553116e-07-2.23076076e-06j
  -1.08943547e-11-6.19505481e-12j ... -8.95017233e-11+7.28635785e-11j
   4.24068556e-16+5.34674036e-17j -5.40435893e-22+4.55896452e-22j]
 [ 1.00000000e+00+6.83769793e-10j -8.62070765e-06+6.72835093e-08j
   4.73611162e-10+2.41736439e-10j ... -1.55527190e-09+2.89631374e-10j
  -5.53689000e-14-7.18999099e-14j -7.66802040e-21+3.47844043e-19j]
 ...
 [-8.91258467e-02-4.35349934e-02j -5.47786249e-02+3.23516810e-01j
  -6.57250070e-03+2.21032911e-02j ...  2.97134808e-02+1.01438073e-02j
   1.

In [12]:
result.results[0]

ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(unitary=[[ 1.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 1.00000000e+00+8.79141144e-11j  3.45553116e-07-2.23076076e-06j
  -1.08943547e-11-6.19505481e-12j ... -8.95017233e-11+7.28635785e-11j
   4.24068556e-16+5.34674036e-17j -5.40435893e-22+4.55896452e-22j]
 [ 1.00000000e+00+6.83769793e-10j -8.62070765e-06+6.72835093e-08j
   4.73611162e-10+2.41736439e-10j ... -1.55527190e-09+2.89631374e-10j
  -5.53689000e-14-7.18999099e-14j -7.66802040e-21+3.47844043e-19j]
 ...
 [-8.91258467e-02-4.35349934e-02j -5.47786249e-02+3.23516810e-01j
  -6.57250070e-03+2.21032911e-02j ...  2.97134808e-02+1.01438073e-02j
   1.63690757e-02-9.79792032e-02j  2.00520607e-03-4.70756124e-03j]
 [-8.91261672e-02-4.35356240e-02j -5.14412895e-02+3.24159429e-01j
  -9.67315097e-03+1.23407149e

In [13]:
backend.solver